In [3]:
# postgress 13 server created and running via docker and forwarded to localhost:5432 (see readme)
# requires sqlalchemy and psycopg2 modules
from sqlalchemy import create_engine
user, password = 'root', 'root'
url, port = 'localhost', 5432
db_name = 'ny_taxi'
engine = create_engine(f'postgresql://{user}:{password}@{url}:{port}/{db_name}')
engine.connect()

In [4]:
import pandas as pd

query = """
SELECT 1 as number;
"""

pd.read_sql(query, con=engine)

,number
0,1


In [5]:

query = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
"""

pd.read_sql(query, con=engine)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity


In [10]:
# to postgress sql
df = pd.read_csv('data/yellow_tripdata_2019-01.csv', nrows=100)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.to_sql(name='yellow_tripdata_trip', con=engine, index=False)

100

In [11]:
# from postgress sql
query = """
SELECT * FROM yellow_tripdata_trip LIMIT 10
"""

pd.read_sql(query, con=engine)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.00,0.3,9.95,None
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.00,0.3,16.30,None
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.00,0.3,5.80,None
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.00,0.3,7.55,None
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.00,0.3,55.55,None
5,2,2018-11-28 16:25:49,2018-11-28 16:28:26,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,5.76,0.3,13.31,None
6,2,2018-11-28 16:29:37,2018-11-28 16:33:43,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.00,0.3,55.55,None
7,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,N,163,229,1,6.5,0.5,0.5,1.25,0.00,0.3,9.05,None
8,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,N,229,7,1,13.5,0.5,0.5,3.70,0.00,0.3,18.50,None
9,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,N,141,234,1,10.0,0.5,0.5,1.70,0.00,0.3,13.00,None


In [1]:
# ingest whole csv in 100k chunks
import time
import pandas as pd

#reset table
db_name = 'yellow_tripdata_trip'
fp = 'data/yellow_tripdata_2019-01.csv'
date_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
df_head = pd.read_csv(fp, nrows=0, parse_dates=date_cols)
df.head.to_sql(name=db_name, con=engine, if_exists='replace')

# itterate over data and upload in batches
df_iter = pd.read_csv(fp, iterator = True, chunksize=100_000, parse_dates=date_cols)
for idx, df in enumerate(df_iter):
    t0 = time()
    nrows = len(df)
    df.to_sql(name=db_name, con=engine, index=False)
    print(f'Batch {idx} ({nrows=}) ingested in {time.time()-t0:.2f} seconds.')

NameError: name 'pd' is not defined